# Инструменты для работы с текстом

Анализ текстовых данных - это отдельное направление, здесь будет совсем небольшое введение.
С текстовыми данными можно решать как задачи обучения с учителем (классификация текстов), так и задачу обучения без учителя (кластеризация).

Предобработка текста

Первый шаг любой аналитики – получение данных. Предположим, что данные представляются собой набор текстов. Все известные нам алгоритмы работают не в текстами, а с объектами, которые описываются вектором признаков (чаще всего численных, категориальные мы умеем преобразовывать). Что делать, если наши объекты - это текст? 

Следующая после получения данных задача: предобработка. Основная цель предобработки: преобразовать текстовые данные в удобный для построения модели вид.

Базовые шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. удаление пунктуации
5. фильтрация по частоте/длине/соответствию регулярному выражению
6. лемматизация или стемминг
7. векторизация (эмбеддинг)

Чаще всего применяются все эти шаги, но в разных задачах какие-то могут опускаться, поскольку приводят к потере информации

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 

* Сейчас мы попробуем получить преобразование предложений в численный вектор, с которым может работать стандартный алгоритм машинного обучения. 
* Для этого нам понадобится познакомиться с понятием n-gram - самых мелких элементов предложения, с которыми можно работать. 
* Подсчитав количество этих n-грам в предложениях, мы получим искомые численные представления.

## n-граммы

Самые мелкие структуры языка, с которыми мы работаем, называются **n-граммами**.
У n-граммы есть параметр n - количество слов, которые попадают в такое представление текста.
* Если n = 1 - то мы смотрим на то, сколько раз каждое слово встретилось в тексте. Получаем _униграммы_
* Если n = 2 - то мы смотрим на то, сколько раз каждая пара подряд идущих слов, встретилась в тексте. Получаем _биграммы_

Функция для работы с n-граммами реализована в библиотке **nltk** (Natural Language ToolKit), импортируем эту функцию: 

In [ ]:
from nltk import ngrams

Прежде чем получать n-граммы, нужно разделить предложение на отдельные слова.  Для этого используем метод ```split()```.

In [ ]:
sentence = 'Кто же победит на выборах в США: Трамп или Байден?'

In [ ]:
sentence_split = sentence.split()
sentence_split

['Кто',
 'же',
 'победит',
 'на',
 'выборах',
 'в',
 'США:',
 'Трамп',
 'или',
 'Байден?']

Кажется, что нам тут мешают знаки препинания. Дайвайте от них избавимся. 

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
for ch in string.punctuation:
  sentence = sentence.replace(ch,"")

In [ ]:
sentence

'Кто же победит на выборах в США Трамп или Байден'

In [ ]:
sentence_split = sentence.split()

In [ ]:
sentence_split

['Кто', 'же', 'победит', 'на', 'выборах', 'в', 'США', 'Трамп', 'или', 'Байден']

Чтобы получить n-грамму для такой последовательности, используем функцию ```ngrams()```. 

На вход передается два параметра:
* лист с разделенным на отдельные слова предложением (у нас он хранится в переменной ```sent```);
* параметр n, определяющий, какой тип n-грамм мы хотим получить.


Чтобы полученный объект отобразить, делаем из него ```list```. 

In [ ]:
list(ngrams(sentence_split, 1)) # униграммы

[('Кто',),
 ('же',),
 ('победит',),
 ('на',),
 ('выборах',),
 ('в',),
 ('США',),
 ('Трамп',),
 ('или',),
 ('Байден',)]

Аналогично мы можем получить биграммы - для этого заменяем параметр **n** в функции **ngrams** с 1 на 2.

In [ ]:
list(ngrams(sentence_split, 2)) # биграммы

[('Кто', 'же'),
 ('же', 'победит'),
 ('победит', 'на'),
 ('на', 'выборах'),
 ('выборах', 'в'),
 ('в', 'США'),
 ('США', 'Трамп'),
 ('Трамп', 'или'),
 ('или', 'Байден')]

In [ ]:
list(ngrams(sentence_split, 3)) # триграммы

[('Кто', 'же', 'победит'),
 ('же', 'победит', 'на'),
 ('победит', 'на', 'выборах'),
 ('на', 'выборах', 'в'),
 ('выборах', 'в', 'США'),
 ('в', 'США', 'Трамп'),
 ('США', 'Трамп', 'или'),
 ('Трамп', 'или', 'Байден')]

In [ ]:
list(ngrams(sentence_split, 5)) # ... пентаграммы

[('Кто', 'же', 'победит', 'на', 'выборах'),
 ('же', 'победит', 'на', 'выборах', 'в'),
 ('победит', 'на', 'выборах', 'в', 'США'),
 ('на', 'выборах', 'в', 'США', 'Трамп'),
 ('выборах', 'в', 'США', 'Трамп', 'или'),
 ('в', 'США', 'Трамп', 'или', 'Байден')]

## Векторизаторы

Векторизатор преобразует слово или набор слов в числовой вектор, понятный алгоритму машинного обучения, который привык работать с числовыми табличными данными.

Ниже - пример преобразования слов в двумерных вектор, каждому слову соответствует точка на плоскости.

<a href="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg
" target="_blank"><img src="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

На начальном этапе нам будет достаточно тех инструментов, которые уже есть в библиотеке **sklearn**.

In [ ]:
from sklearn.tree import DecisionTreeClassifier # можно заменить на другой классификатор
from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов", см. далее

Самый простой способ извлечь признаки из текстовых данных -- векторизаторы: `CountVectorizer` и `TfidfVectorizer`

Объект `CountVectorizer` делает следующую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

<a href="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg
" target="_blank"><img src="https://drive.google.com/uc?id=1jHmkrGZTMawM46Yzxh243Ur1y5pYKzrl" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

На рисунке пример векторизации для униграмм, но можно использовать любые n-граммы. Для этого у объекта ```CountVectorizer()``` есть параметр **ngram_range**, который отвечает за то, какие n-граммы мы используем в качестве признаов:<br/>
ngram_range=(1, 1) -- униграммы<br/>
ngram_range=(3, 3) -- триграммы<br/>
ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

## Пример

К сожалению, на русском языке всё ещё очень мало годных наборов данных. Набор данных нашёл тут: https://github.com/sismetanin/rureviews

In [ ]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1j-DhO_XD5EqzOqVSR4Wz1kNzhLmpeTkZ"}) 
downloaded.GetContentFile('women-clothing-accessories.csv')

In [ ]:
data = pd.read_csv('women-clothing-accessories.csv', sep='\t', usecols=[0, 1])

In [ ]:
data.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [ ]:
data.sample(10)

,review,sentiment
9700,"товар не пришел, деньги не вернули. продавец в...",negative
78254,"куртка пришла быстро, хорошего качества и без ...",positive
69048,"Очень теплый кардиган, крупная вязка, ощущаетс...",positive
29445,"заказала размер М, по факту размер S. Рукава к...",negative
27592,"товар не пришел, деньги вернули",negative
25837,"Мятый, не держит, получила быстро",negative
26702,Качество пошива нормальное. Но!!! Пуховик выпо...,negative
74390,Платье прикольное! Шло до Свердл.обл. ровно 1 ...,positive
36988,"товар не пришел,вернули деньги.",neautral
15496,Очень криво сшито. одна половина длиннее другой,negative


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)

In [ ]:
data.sentiment.value_counts()

neautral    30000
negative    30000
positive    30000
Name: sentiment, dtype: int64

In [ ]:
y_train.value_counts()

negative    21034
positive    21019
neautral    20946
Name: sentiment, dtype: int64

Инициализируем `CountVectorizer()`, указав в качестве признаков униграммы:

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 1))

После инициализации _vectorizer_ можно обучить на наших данных. 

Для обучения используем обучающую выборку ```x_train```, но в отличие от классификатора мы используем метод ```fit_transform()```: сначала обучаем наш векторизатор, а потом сразу применяем его к нашему набору данных. Это похоже на то, как мы работали с one-hot-encoderом.

In [ ]:
vectorized_x_train = vectorizer.fit_transform(x_train)

Так как результат не зависит от порядка слов в текстах, то говорят, что такая модель представления текстов в виде векторов получается из *гипотезы представления текста как мешка слов*

В `vectorizer.vocabulary_` лежит словарь, отображение слов в их индексы:

In [ ]:
list(vectorizer.vocabulary_.items())[:10]

[('пришёл', 31551),
 ('мал', 18011),
 ('материал', 18269),
 ('ожидался', 23477),
 ('другой', 11224),
 ('немного', 20996),
 ('прозрачный', 32010),
 ('товар', 39618),
 ('пришел', 31484),
 ('менее', 18455)]

In [ ]:
vectorized_x_train.shape

(62999, 44490)

In [ ]:
x_train.shape

(62999,)

Так как теперь у нас есть **численное представление** и набор входных признаков, то мы можем обучить нашу модель

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(vectorized_x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
clf2 = MultinomialNB()
clf2.fit(vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

С тестовыми данными нужно проделать то же самое, что и с данными для обучения: сделать из текстов вектора, которые можно передавать в классификатор для прогноза класса объекта. 

У нас уже есть обученный векторизатор ```vectorizer```, поэтому используем метод ```transform()``` (просто применить его), а не ```fit_transform``` (обучить и применить).

In [ ]:
vectorized_x_test = vectorizer.transform(x_test)

Как раньше, для получения прогноза у обученного классификатора используем метод ```predict()```.

С помощью функции ```classification_report()```, которая считает сразу несколько метрик качества классификации, посмотрим на то, насколько хорошо мы предсказываем положительную или отрицательную тональность твита .

In [ ]:
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.52      0.53      0.52      9054
    negative       0.63      0.63      0.63      8966
    positive       0.76      0.75      0.76      8981

    accuracy                           0.64     27001
   macro avg       0.64      0.64      0.64     27001
weighted avg       0.64      0.64      0.64     27001



In [ ]:
pred2 = clf2.predict(vectorized_x_test)
print(classification_report(y_test, pred2))

              precision    recall  f1-score   support

    neautral       0.59      0.66      0.62      9054
    negative       0.72      0.62      0.67      8966
    positive       0.84      0.85      0.84      8981

    accuracy                           0.71     27001
   macro avg       0.72      0.71      0.71     27001
weighted avg       0.72      0.71      0.71     27001



Итак, наивный байесовский классификатор легко побил дерево решений. Дальше работаем с ним.

### Отступление: F-мера

Прошлый раз мы разобрали метрики качества классификации, которые выводятся из матрицы ошибок (confision matrix). 

**Полнота** (Sensitivity, True Positive Rate, Recall, Hit Rate) отражает какой процент объектов положительного класса мы правильно классифицировали.

**Точность** (Precision, Positive Predictive Value) отражает какой процент положительных объектов (т.е. тех, что мы считаем положительными) правильно классифицирован. (Не путать с Accuracy!)

Легко построить алгоритм со 100%-й полнотой: он все объекты относит к классу 1, но при этом точность может быть очень низкой. Нетрудно построить алгоритм с близкой к 100% точностью: он относит к классу 1 только те объекты, в которых уверен, при этом полнота может быть низкая.

**F1-мера** (F1 score) является средним гармоническим точности и полноты, максимизация этого функционала приводит к одновременной максимизации этих двух «ортогональных критериев»

$$F_1 = \frac{2}{\mathrm{recall}^{-1} + \mathrm{precision}^{-1}} = 2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}} = \frac{\mathrm{tp}}{\mathrm{tp} + \frac12 (\mathrm{fp} + \mathrm{fn}) } $$

Также рассматривают весовое среднее гармоническое точности и полноты –  $F_\beta$-меру:

$$F_\beta = (1 + \beta^2) \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{(\beta^2 \cdot \mathrm{precision}) + \mathrm{recall}} = \frac {(1 + \beta^2) \cdot \mathrm{tp} }{(1 + \beta^2) \cdot \mathrm{tp} + \beta^2 \cdot \mathrm{fn} + \mathrm{fp}}\,$$

Изменение $\beta$ позволяет делать один из критериев (точность или полноту) важнее при оптимизации.

## Биграммы

Попробуем сделать то же самое, используя в качестве признаков униграммы и биграммы:

In [ ]:
# инициализируем векторайзер 
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))

In [ ]:
# обучаем его и сразу применяем к x_train
bigram_vectorized_x_train = bigram_vectorizer.fit_transform(x_train)

In [ ]:
# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(bigram_vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# применяем обученный векторизатор к тестовым данным
bigram_vectorized_x_test = bigram_vectorizer.transform(x_test)

In [ ]:
# получаем предсказания и выводим информацию о качестве
pred = clf.predict(bigram_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.61      0.67      0.64      9054
    negative       0.72      0.66      0.69      8966
    positive       0.87      0.86      0.87      8981

    accuracy                           0.73     27001
   macro avg       0.74      0.73      0.73     27001
weighted avg       0.74      0.73      0.73     27001



У меня получилось повысить точность на пару процентов по сравнению с униграммами

In [ ]:
bigram_vectorized_x_train.shape

(62999, 461507)

"Признаков" объектов стало на порядок больше.

## Токенизация

Токенизировать - значит, поделить текст на части: слова, ключевые слова, фразы, символы и т.д., иными словами **токены**.

Самый наивный способ токенизировать текст - разделить с помощью функции `split()`. Но `split` упускает очень много всего, например, не отделяет пунктуацию от слов. Кроме этого, есть ещё много менее тривиальных проблем, поэтому лучше использовать готовые токенизаторы.

In [ ]:
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk

Чтобы использовать токенизатор ```word_tokenize```, нужно сначала скачать данные для nltk о пунктуации и стоп-словах.

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Применим токенизацию:

In [ ]:
sentence = 'Кто же победит на выборах в США: Трамп или Байден?'
word_tokenize(sentence)

['Кто',
 'же',
 'победит',
 'на',
 'выборах',
 'в',
 'США',
 ':',
 'Трамп',
 'или',
 'Байден',
 '?']

Сравните с использованием ```split()```:

In [ ]:
sentence.split()

['Кто',
 'же',
 'победит',
 'на',
 'выборах',
 'в',
 'США:',
 'Трамп',
 'или',
 'Байден?']

В nltk вообще есть довольно много токенизаторов:

In [ ]:
from nltk import tokenize
dir(tokenize)[:16]

['BlanklineTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'PunktSentenceTokenizer',
 'RegexpTokenizer',
 'ReppTokenizer',
 'SExprTokenizer',
 'SpaceTokenizer',
 'StanfordSegmenter',
 'TabTokenizer',
 'TextTilingTokenizer',
 'ToktokTokenizer',
 'TreebankWordTokenizer',
 'TweetTokenizer',
 'WhitespaceTokenizer',
 'WordPunctTokenizer']

Одни умеют выдавать индексы в строке для начала и конца каждого слова-токена:

In [ ]:
wh_tok = tokenize.WhitespaceTokenizer()
list(wh_tok.span_tokenize(sentence))

[(0, 3),
 (4, 6),
 (7, 14),
 (15, 17),
 (18, 25),
 (26, 27),
 (28, 32),
 (33, 38),
 (39, 42),
 (43, 50)]

Некторые токенизаторы ведут себя специфично:

In [ ]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

['do', "n't", 'stop', 'me']

А некоторые -- вообще не для текста на естественном языке:

In [ ]:
tokenize.SExprTokenizer().tokenize("(a (b c)) d e (f)")

['(a (b c))', 'd', 'e', '(f)']

**Правильный токенизатор подбирается исходя из требований задачи!**

## Стоп-слова

**Стоп-слова** - это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе. Для модели это просто шум. А шум нужно убирать. По аналогичной причине убирают и пунктуацию.

In [ ]:
# импортируем стоп-слова из библиотеки nltk
from nltk.corpus import stopwords

# посмотрим на стоп-слова для русского языка
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [ ]:
noise = stopwords.words('russian')

Теперь нужно обучать нашу модель с учетом новых знаний про токенизацию и стоп-слова. 

Для этого мы можем собрать новый векторизатор, передав ему на вход:
* какие n-граммы нам нужны, параметр **ngram_range**;
* какой токенизатор мы используем, параметр **tokenizer**;
* какие у нас стоп-слова, параметр **stop_words**.

In [ ]:
# инициализируем умный векторайзер 
smart_vectorizer = CountVectorizer(ngram_range=(1, 1), stop_words=noise)

In [ ]:
# обучаем его и сразу применяем к x_train
smart_vectorized_x_train = smart_vectorizer.fit_transform(x_train)

In [ ]:
smart_vectorized_x_train.shape

(62999, 44348)

In [ ]:
list(smart_vectorizer.vocabulary_.items())[:10]

[('пришёл', 31452),
 ('мал', 17955),
 ('материал', 18213),
 ('ожидался', 23391),
 ('немного', 20921),
 ('прозрачный', 31910),
 ('товар', 39502),
 ('пришел', 31385),
 ('менее', 18398),
 ('недели', 20604)]

In [ ]:
# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(smart_vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# применяем обученный векторайзер к тестовым данным
smart_vectorized_x_test = smart_vectorizer.transform(x_test)

In [ ]:
# получаем предсказания и выводим информацию о качестве
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.58      0.65      0.61      9054
    negative       0.71      0.61      0.66      8966
    positive       0.82      0.84      0.83      8981

    accuracy                           0.70     27001
   macro avg       0.70      0.70      0.70     27001
weighted avg       0.70      0.70      0.70     27001



Получилось чуть хуже. 

Что ещё можно сделать?

## Лемматизация

**Лемматизация** – это сведение разных форм одного слова к начальной форме – **лемме**. Почему это хорошо?
* Во-первых, естественно рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.
* Во-вторых, некоторые стоп-слова стоят только в начальной форме, и без лематизации выкидываем мы только её.

Для русского есть хороший лемматизатор pymorphy. 

Стемминг (англ. stemming — находить происхождение) — это процесс нахождения основы слова для заданного исходного слова. Основа слова не обязательно совпадает с морфологическим корнем слова. 

### [Pymorphy](http://pymorphy2.readthedocs.io/en/latest/)
Это модуль на питоне, довольно быстрый и с кучей функций.

In [ ]:
# устанавливаем pymorphy2
!pip install pymorphy2

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 8.2MB 4.9MB/s 


В pymorphy2 для морфологического анализа слов есть ```MorphAnalyzer()```:

In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

pymorphy2 работает с отдельными словами. Если дать ему на вход предложение - он его просто не лемматизирует, т.к. не понимает:

In [ ]:
sentence = 'Кто же победит на выборах в США: Трамп или Байден?'
sent = word_tokenize(sentence)

Лемматизируем слово "победит" из предложения ```sentence``` с помощью метода ```parse()```:

In [ ]:
ana = pymorphy2_analyzer.parse(sent[2])
ana

[Parse(word='победит', tag=OpencorporaTag('VERB,perf,tran sing,3per,futr,indc'), normal_form='победить', score=0.846153, methods_stack=((DictionaryAnalyzer(), 'победит', 2483, 9),)),
 Parse(word='победит', tag=OpencorporaTag('NOUN,inan,masc sing,nomn'), normal_form='победит', score=0.076923, methods_stack=((DictionaryAnalyzer(), 'победит', 34, 0),)),
 Parse(word='победит', tag=OpencorporaTag('NOUN,inan,masc sing,accs'), normal_form='победит', score=0.076923, methods_stack=((DictionaryAnalyzer(), 'победит', 34, 3),))]

Выведем его нормальную форму:

In [ ]:
ana[0].normal_form

'победить'

Нормализация предложения "вижу три села" может дать "видеть тереть сесть"

In [ ]:
sent2 = word_tokenize('вижу три села')

In [ ]:
ana2 = pymorphy2_analyzer.parse(sent2[2])
ana2[0].normal_form

'село'

## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений выдает **tf-idf** каждого слова.

Как считается tf-idf:

**TF (term frequency)** – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_{t}}{\sum_k n_{k}} $$

**IDF (inverse document frequency)** – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

Перемножаем их:
$$TFIDF(t, d, D) = TF(t,d) \times IDF(i, D)$$

Cмысл: если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Действуем аналогично, как с ```CountVectorizer()```:

In [ ]:
# инициализируем векторизатор, в качестве переменных используем униграммы
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

In [ ]:
# обучаем его и сразу применяем к x_train
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)

In [ ]:
# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(tfidf_vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# применяем обученный векторизатор к тестовым данным
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.59      0.67      0.63      9054
    negative       0.72      0.63      0.67      8966
    positive       0.84      0.84      0.84      8981

    accuracy                           0.71     27001
   macro avg       0.72      0.71      0.71     27001
weighted avg       0.72      0.71      0.71     27001



Иногда пунктуация бывает и не шумом - главное отталкиваться от задачи. Что будет если вообще не убирать пунктуацию? На примере с твитами хорошо было бы видно, что пунктуация работает хорошо ))

In [ ]:
# инициализируем умный векторайзер stop-words НЕ ИСПОЛЬЗУЕМ!
alternative_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), 
                                               tokenizer=word_tokenize)

# обучаем его и сразу применяем к x_train
alternative_tfidf_vectorized_x_train = alternative_tfidf_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(alternative_tfidf_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
alternative_tfidf_vectorized_x_test = alternative_tfidf_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(alternative_tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.59      0.67      0.63      9054
    negative       0.72      0.64      0.68      8966
    positive       0.85      0.84      0.84      8981

    accuracy                           0.71     27001
   macro avg       0.72      0.71      0.72     27001
weighted avg       0.72      0.71      0.72     27001



Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [ ]:
cool_token = 'плохо'
pred = ['positive' if cool_token in review else 'negative' for review in x_test]
print(classification_report(pred, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    neautral       0.00      0.00      0.00         0
    negative       0.94      0.33      0.49     25808
    positive       0.02      0.14      0.03      1193

    accuracy                           0.32     27001
   macro avg       0.32      0.16      0.17     27001
weighted avg       0.90      0.32      0.47     27001



## Символьные n-граммы

В некоторых задачах в качестве признаков могут быть использщованы, n-граммы символов. Для этого необходимо установить в ```CountVectorizer()``` параметр ```analyzer = 'char'```, то есть анализировать символы.

In [ ]:
# инициализируем векторайзер для символов
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 6))

# обучаем его и сразу применяем к x_train
char_vectorized_x_train = char_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(char_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
char_vectorized_x_test = char_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(char_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.59      0.71      0.64      9054
    negative       0.73      0.62      0.67      8966
    positive       0.87      0.83      0.85      8981

    accuracy                           0.72     27001
   macro avg       0.73      0.72      0.72     27001
weighted avg       0.73      0.72      0.72     27001



Cимвольные n-граммы используются, например, для задачи определения языка. Ещё одна замечательная особенность признаков-символов - для них не нужна токенизация и лемматизация, можно использовать такой подход для языков, у которых нет готовых анализаторов.